## Import classes, libraries

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect
from src.storage.petastorm_storage_engine import PetastormStorageEngine
from src.readers.opencv_reader import OpenCVReader
from src.catalog.models.df_metadata import DataFrameMetadata
from src.storage.abstract_storage_engine import AbstractStorageEngine
from src.storage.ffmpeg_storage_engine import FFmpegStorageEngine
from src.catalog.models.df_column import DataFrameColumn
from src.models.storage.batch import Batch
from src.catalog.column_type import ColumnType, NdArrayType
from src.readers.abstract_reader import AbstractReader
from src.readers.petastorm_reader import PetastormReader
from typing import Iterator
import os
import ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
one_min_video = "/home/akshay/Downloads/pallof.mp4"
thirty_sec_video = "/home/akshay/Downloads/30secseinfeld.mkv"

In [18]:
%%time
opencv_reader = OpenCVReader(one_min_video, 30000000)
ffengine = FFmpegStorageEngine()
seinfeldtable = DataFrameMetadata("oneminseinfeld", "seinfeldvideo")
seinfeldtable.resolution = (1280, 720)
ffengine.create(seinfeldtable)
ffengine.open_write(seinfeldtable)
for batch in opencv_reader.read():
    ffengine.write(seinfeldtable, batch)
ffengine.close_write(seinfeldtable)

Closed
CPU times: user 11.5 s, sys: 1.69 s, total: 13.2 s
Wall time: 48.2 s


In [3]:
ffengine = FFmpegStorageEngine()

In [4]:
def sequence(start, end, step):
    seinfeldtable = DataFrameMetadata("oneminseinfeld", "seinfeldvideo")
    seinfeldtable.resolution = (1280, 720)
    
    results = []
    
    for i in range(start, end, step):
        start_time = time.time()
        
        ffengine.open_read(seinfeldtable, ("between", (0, i)))
        for images in ffengine.read(seinfeldtable):
            image_arrays = images.frames['data'].values
        ffengine.close_read(seinfeldtable)
        
        end_time = time.time()
        
        results.append(end_time - start_time)
    
    return results

In [9]:
start, end, step = 0, 1000, 50

n = len(range(start, end, step))

result_sum = [0] * n

warmup_rounds = 1
experiment_rounds = 2

ffengine.process = None

for i in range(warmup_rounds):
    sequence(start, end, step)

for i in range(experiment_rounds):
    test_i_results = sequence(start, end, step)
    result_sum = [result_sum[i] + test_i_results[i] for i in range(n)]

average_result = [x / experiment_rounds for x in result_sum]

print(average_result)

[3.4130364656448364, 3.7089719772338867, 4.1887887716293335, 4.5869057178497314, 5.232674717903137, 6.480870604515076, 5.946494221687317, 6.432538986206055, 6.894888758659363, 6.873034715652466, 7.413555383682251, 7.740192532539368, 8.630574822425842, 8.65120542049408, 9.326254963874817, 9.777140855789185, 9.880618214607239, 9.979219794273376, 10.747848510742188, 11.049229264259338]


In [53]:
n_frames = int(ffmpeg.probe("ffmpeg_data/seinfeldvideo.mp4")['streams'][0]['nb_frames'])